In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
import torchvision

/home/briancy2/.conda/envs/iwildcam/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
META_DIR = "data/metadata/metadata/"
TRAIN_DIR = "data/train/train/"
TEST_DIR = "data/test/test/"

test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))
train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))

#Test images
df_test = pd.DataFrame({'id': [item['id'] for item in test_data['images']],
                                'seq_id': [item['seq_id'] for item in test_data['images']],
                                'file_name': [item['file_name'] for item in test_data['images']],
                                'location': [item['location'] for item in test_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in test_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in test_data['images']]})
#Train images
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['images']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})
# Detection for train test
detections = json.load(open(META_DIR+"iwildcam2022_mdv4_detections.json"))['images']
det_dict = dict()
for detection in tqdm(detections):
    det_dict[detection['file']] = detection['detections']
df_detection = pd.DataFrame({'file': [item['file'] for item in detections],
                                'detections': [item['detections'] for item in detections]})
# Test sequence ids
test_sequence_ids = pd.unique(df_test['seq_id'])
# Train sequence id and count
train_seq_count = pd.read_csv(META_DIR+"train_sequence_counts.csv")

100%|██████████| 261428/261428 [00:00<00:00, 1483592.67it/s]


In [3]:
train_seq_count.

SyntaxError: invalid syntax (1790939137.py, line 1)

In [5]:
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine", 0.2, 100)
file = open('submission_deep8.txt', 'w')
file.write("Id,Predicted")
for sequence_id in tqdm(test_sequence_ids):
    tracker = Tracker(metric)
    results = []
    img_rows = df_test.loc[df_test.seq_id == sequence_id]
    for index, img_row in img_rows.iterrows():
        detections = df_detection.loc[df_detection.file == ('test/'+img_row['file_name'])]['detections'].to_list()[0]
        #print(set([d["category"] for d in detections]))
        detections = [Detection(tuple(det["bbox"]), det["conf"], []) for det in detections if det['conf']]
        
#         boxes = np.array([d for d in filtered_det])
#         scores = np.array([d.confidence for d in detections])
        tracker.predict()
        tracker.update(detections)
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            results.append(track.track_id)
    count = len(set(results))   
    file.write('\n' + str(sequence_id) + ',' + str(count))

100%|██████████| 11028/11028 [19:35<00:00,  9.38it/s]


In [7]:
file.close()

In [3]:
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
print(len(train_seq_count))
file = open('submission_deep8.txt', 'w')
file.write("Id,Predicted")
counts = []
metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine", 0.2, 100)
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    tracker = Tracker(metric)
    results = []
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    for index, img_row in img_rows.iterrows():
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        detections = [Detection(tuple(det["bbox"]), det["conf"], []) for det in detections]
#         boxes = np.array([d for d in filtered_det])
#         scores = np.array([d.confidence for d in detections])
        tracker.predict()
        tracker.update(detections)
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            results.append(track.track_id)
    count = len(set(results))   
    counts.append(count)
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))
    

1780


1780it [09:14,  3.21it/s]


In [6]:
img_rows = df_train.loc[df_train.seq_id == '302d5988-7d42-11eb-8fb5-0242ac1c0002']


In [7]:
loss = np.mean((np.array(counts) - train_seq_count["count"].to_numpy()))

In [8]:
print(loss)

0.06685393258426967


In [9]:
for index, img_row in img_rows.iterrows():
    detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
    print([det["conf"] for det in detections])

[0.998, 0.998, 0.988, 0.98, 0.951, 0.693, 0.38, 0.202, 0.202, 0.2, 0.174]
[0.998, 0.992, 0.989, 0.989, 0.984, 0.965, 0.955, 0.375, 0.333, 0.271, 0.171, 0.162]
[0.998, 0.993, 0.993, 0.977, 0.952, 0.947, 0.85, 0.668, 0.476, 0.353, 0.337, 0.247, 0.176, 0.169, 0.123, 0.121, 0.104]
[0.999, 0.998, 0.995, 0.993, 0.977, 0.965, 0.774, 0.138, 0.132, 0.126, 0.104]
[0.998, 0.996, 0.989, 0.988, 0.984, 0.969, -0.879, 0.733, 0.615, 0.353, 0.315, 0.277, 0.159, 0.159, 0.13, 0.124, 0.111]
[0.998, 0.998, 0.996, 0.987, 0.986, 0.978, -0.92, 0.506, 0.362, 0.198, 0.197, 0.184, 0.164, 0.103]
[0.999, 0.997, 0.994, 0.992, 0.959, -0.899, 0.64, 0.621, 0.464, 0.374, 0.364, 0.118, 0.109]


In [7]:
img_rows

,id,seq_id,file_name,location,seq_num_frames,seq_frame_num
77815,94a57b08-21bc-11ea-a13a-137349068a90,302d5988-7d42-11eb-8fb5-0242ac1c0002,94a57b08-21bc-11ea-a13a-137349068a90.jpg,170,7,0
77816,97a3447a-21bc-11ea-a13a-137349068a90,302d5988-7d42-11eb-8fb5-0242ac1c0002,97a3447a-21bc-11ea-a13a-137349068a90.jpg,170,7,1
77817,924d3828-21bc-11ea-a13a-137349068a90,302d5988-7d42-11eb-8fb5-0242ac1c0002,924d3828-21bc-11ea-a13a-137349068a90.jpg,170,7,2
77818,8be47ab4-21bc-11ea-a13a-137349068a90,302d5988-7d42-11eb-8fb5-0242ac1c0002,8be47ab4-21bc-11ea-a13a-137349068a90.jpg,170,7,3
77819,8e8e4e02-21bc-11ea-a13a-137349068a90,302d5988-7d42-11eb-8fb5-0242ac1c0002,8e8e4e02-21bc-11ea-a13a-137349068a90.jpg,170,7,4
77820,8e63e9fa-21bc-11ea-a13a-137349068a90,302d5988-7d42-11eb-8fb5-0242ac1c0002,8e63e9fa-21bc-11ea-a13a-137349068a90.jpg,170,7,5
77821,8a5f4480-21bc-11ea-a13a-137349068a90,302d5988-7d42-11eb-8fb5-0242ac1c0002,8a5f4480-21bc-11ea-a13a-137349068a90.jpg,170,7,6


In [4]:
file.close()

In [ ]:
!git add .

In [37]:
!git commit -am "created deep_sort_predictions notebook"

[main 7789cd0] created deep_sort_predictions notebook
 3 files changed, 318 insertions(+), 9 deletions(-)
 create mode 100644 .ipynb_checkpoints/deep_sort_predictions-checkpoint.ipynb
 create mode 100644 deep_sort_predictions.ipynb
